In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('covid_toy.csv')

In [3]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [5]:
df.duplicated().sum()

1

In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [8]:
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2,random_state=10)

In [9]:
x_train.shape

(80, 5)

In [10]:
#Handling the missing values in fever column

si=SimpleImputer()
x_train_fever=si.fit_transform(x_train[['fever']])
x_test_fever=si.transform(x_test[['fever']])

In [11]:
df.cough.unique()

array(['Mild', 'Strong'], dtype=object)

In [12]:
#Encoding the ordinal cough column

ode=OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough=ode.fit_transform(x_train[['cough']])
x_test_cough=ode.transform(x_test[['cough']])

In [13]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [14]:
# Encoding the Nominal(gender and city) columns

ohe=OneHotEncoder(drop='first',sparse=False)
x_train_city_gender=ohe.fit_transform(x_train[['city','gender']])
x_test_city_gender=ohe.transform(x_test[['city','gender']])

In [15]:
x_train_city_gender[:5]

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 1.],
       [1., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [16]:
x_train_age=x_train.drop(columns=['gender','fever','cough','city']).values
x_test_age=x_test.drop(columns=['gender','fever','cough','city']).values

In [17]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_city_gender,x_train_cough),axis=1)
x_test_transformed=np.concatenate((x_test_age,x_test_fever,x_test_city_gender,x_test_cough),axis=1)                                       

## Using Column Transformer

In [18]:
from sklearn.compose import ColumnTransformer

In [19]:
ct = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first',sparse=False),['gender','city'])

],remainder='passthrough')

In [20]:
x_train_CTransformed=ct.fit_transform(x_train)
x_train_CTransformed.shape
x_train_CTransformed[:5]

array([[ 99.,   1.,   0.,   1.,   0.,   0.,  59.],
       [101.,   1.,   0.,   1.,   0.,   0.,  68.],
       [101.,   0.,   1.,   1.,   0.,   0.,  19.],
       [102.,   1.,   1.,   1.,   0.,   0.,  20.],
       [100.,   0.,   1.,   0.,   0.,   0.,  80.]])

In [21]:
ct.transform(x_test)[:5]

array([[100.65277778,   1.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  42.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  51.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  22.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  55.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  51.        ]])